In [1]:
NEO4J_URI='neo4j+s://955cd91b.databases.neo4j.io'
NEO4J_USERNAME='neo4j'
NEO4J_PASSWORD='WygIPlQMGaBqzxGa4lwgYoMueAt6gEbe0ZWTbY3NCds'

In [2]:
import os
os.environ['NEO4J_URI']=NEO4J_URI
os.environ['NEO4J_USERNAME']=NEO4J_USERNAME
os.environ['NEO4J_PASSWORD']=NEO4J_PASSWORD

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api=os.getenv('GROQ_API_KEY')
from langchain_groq import ChatGroq
llm=ChatGroq(groq_api_key=groq_api,model_name='Gemma2-9b-It')
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000023B356C9520>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000023B356CA030>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [4]:
moview_query="""
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row
MERGE(m:Movie{id:row.movieId})
SET m.released=date(row.released),
    m.title=row.title,
    m.imdbRating=toFloat(row.imdbRating)
FOREACH (director in split(row.director,'|')|
    MERGE (p:Person{name:trim(director)})
    MERGE(p)-[:DIRECTED]->(m))
FOREACH(actor in split(row.actors,'|') |
    MERGE(p:Person {name:trim(actor)})
    MERGE(p)-[:ACTED_IN]->(m))
FOREACH(genre in split(row.genes,'|')|
    MERGE(g:Genre {name:trim(genre)})
    MERGE(m)-[:IN_GENRE]->(g))
"""


In [5]:
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph(url=NEO4J_URI,username=NEO4J_USERNAME,password=NEO4J_PASSWORD)
graph.query(moview_query)

[]

In [6]:
from langchain.chains import GraphCypherQAChain
chain=GraphCypherQAChain.from_llm(graph=graph,llm=llm,exclude_types=['Genre'],verbose=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x0000023B3D141520>, cypher_generation_chain=LLMChain(prompt=PromptTemplate(input_variables=['question', 'schema'], template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000023B356C9520>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000023B356CA030>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [8]:
chain.graph_schema

'Node properties are the following:\nCEO {YOB: INTEGER, NAME: STRING},Company {name: STRING},enterpreneur {YOB: INTEGER, name: STRING, POB: STRING},country {name: STRING},Person {name: STRING, born: INTEGER},Movie {title: STRING, released: INTEGER, id: STRING, imdbRating: FLOAT}\nRelationship properties are the following:\n\nThe relationships are the following:\n(:Person)-[:DIRECTED]->(:Movie),(:Person)-[:ACTED_IN]->(:Movie)'

In [9]:
examples = [
    {
        'question': 'How many artists are there?',
        'query': 'MATCH (a:Person)-[ACTED_IN]->(:Movie) RETURN count(DISTINCT a)',
    },
    {
        'question': 'Which actors played in the movie Casino?',
        'query': 'MATCH(m:Movie {title:"Casino"})<-[:ACTED_IN]-(a) RETURN a.name',
    },
    {
        'question': 'How many movies has Tom Hanks acted in?',
        'query': 'MATCH(a:Person {name:"Tom Hanks"})-[:ACTED_IN]->(m:Movie) RETURN count(m)',
    },
    {
        'question': 'What are the names of all movies directed by Steven Spielberg?',
        'query': 'MATCH (d:Person {name:"Steven Spielberg"})-[:DIRECTED]->(m:Movie) RETURN m.title',
    },
    {
        'question': 'Which movies were released in 1995?',
        'query': 'MATCH (m:Movie {year:1995}) RETURN m.title',
    },
    {
        'question': 'How many directors have directed more than 5 movies?',
        'query': 'MATCH (d:Person)-[:DIRECTED]->(m:Movie) WITH d, count(m) as movies WHERE movies > 5 RETURN count(d)',
    },
    {
        'question': 'Which actors have acted in both dramas and comedies?',
        'query': 'MATCH (a:Person)-[:ACTED_IN]->(m1:Movie {genre:"Drama"}), (a)-[:ACTED_IN]->(m2:Movie {genre:"Comedy"}) RETURN DISTINCT a.name',
    }
]



In [13]:
response=chain.invoke({'query':'How many artists are there'})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person) RETURN count(p)
Full Context:
[{'count(p)': 1241}]

> Finished chain.


{'query': 'How many artists are there', 'result': 'There are 1241 artists. \n'}

In [23]:
from langchain_core.prompts import FewShotPromptTemplate,PromptTemplate
example_prompt=PromptTemplate.from_template(
    'User Input:{question}\n Cypher query:{query}'
)

prompt=FewShotPromptTemplate(
    examples=examples[:5],
    example_prompt=example_prompt,
    prefix='You are a neo4j expert.Given an input question,create a syntactically very accurate cypher queries',
    suffix='User input:{question}\n Cypher query:',
    input_variables=['question','schema']
)

In [24]:
prompt

FewShotPromptTemplate(input_variables=['question'], examples=[{'question': 'How many artists are there?', 'query': 'MATCH (a:Person)-[ACTED_IN]->(:Movie) RETURN count(DISTINCT a)'}, {'question': 'Which actors played in the movie Casino?', 'query': 'MATCH(m:Movie {title:"Casino"})<-[:ACTED_IN]-(a) RETURN a.name'}, {'question': 'How many movies has Tom Hanks acted in?', 'query': 'MATCH(a:Person {name:"Tom Hanks"})-[:ACTED_IN]->(m:Movie) RETURN count(m)'}, {'question': 'What are the names of all movies directed by Steven Spielberg?', 'query': 'MATCH (d:Person {name:"Steven Spielberg"})-[:DIRECTED]->(m:Movie) RETURN m.title'}, {'question': 'Which movies were released in 1995?', 'query': 'MATCH (m:Movie {year:1995}) RETURN m.title'}], example_prompt=PromptTemplate(input_variables=['query', 'question'], template='User Input:{question}\n Cypher query:{query}'), suffix='User input:{question}\n Cypher query:', prefix='You are a neo4j expert.Given an input question,create a syntactically very ac

In [27]:
# print(prompt.format(question='How many artists are there?',schema='foo'))

In [26]:
llm
chain=GraphCypherQAChain.from_llm(graph=graph,llm=llm,cypher_prompt=prompt,verbose=True)